In [1]:
import pandas as pd
from datetime import timedelta

In [2]:
# Config:
month = 'Jun23'
file_name = 'Uk_Jan_Jun_23'

In [3]:
# Read CSV with Decimals as comma:
df = pd.read_csv(f"./Inputs/{month}/{file_name}.csv", decimal=',')

In [4]:
df.head(3)

,Company (code+name),Year,Delivery note date,Customer code,Customer name,Segment code (customer),Segment name (customer),Global CoFX Amount (net),Sales order number,Order origin
0,141 - CERTIKIN INTERNATIONAL LIMITED,2023,03/01/23 0:00,306,BUCKLAND POOL & BUILDING COMPANY LTD,15.0,POOL SPECIALIST-Instal/Builder,849.73,580963,25
1,141 - CERTIKIN INTERNATIONAL LIMITED,2023,03/01/23 0:00,615,FREEDOM LEISURE !,15.0,POOL SPECIALIST-Instal/Builder,75.02,580883,25
2,141 - CERTIKIN INTERNATIONAL LIMITED,2023,03/01/23 0:00,615,FREEDOM LEISURE !,15.0,POOL SPECIALIST-Instal/Builder,14.6,HAZARDOUS,25


In [5]:
# Renaming Columns:
df = df.rename(columns={'Delivery note date': 'Purchase Date',
                        'Customer Name': 'Customer name', 
                        'Global CoFX Amount (net)': 'Global Amount'})

In [6]:
df["Customer name"].unique()

array(['BUCKLAND POOL & BUILDING COMPANY LTD', 'FREEDOM LEISURE  !',
       'UK POOL STORE LTD', 'LEISURE POOLS LTD', 'POOL SUPPLIES LTD',
       'HEAT PUMPS 4 POOLS LTD', 'PACIFIC POOLS LTD T/A POOL TRADE COUNT',
       'JB ELITE SERVICES LTD', 'AZURE POOL SERVICES LTD',
       'PENINSULA POOLS LTD', 'ASHER SWIMPOOL CENTRE LTD',
       'PBS POOLS LTD', 'RS POOLS LTD',
       'OXFORD POOLS AND HOT TUBS LIMITED', 'SAFFRON SWIMMING POOLS LTD',
       'CORNISH HOT TUBS LIMITED', 'W D S LIMITED!', 'B.T.U SUPPLIES LTD',
       'SPRUCE SERVICES LIMITED', 'AQUA PLATINUM SERVICING LTD',
       'LETTS SWIMMING POOLS LIMITED', 'POOLWORX LIMITED',
       'SWIMMING POOLS (MIDLANDS)', 'TRACIE SKINNER T/AS PUREPOOLS',
       'WENSUM POOLS LTD', 'JML POOLS LTD', 'OCEANOS POOLS LTD !',
       'POOL COVER SERVICING!', 'HTP INC LIMITED', 'KB POOLS LTD',
       'LANGLEY POOL SERVICES LIMITED',
       'MERLIN POOLS AND SPAS LLP (COLCHESTER)',
       'TOPLINE ELECTRONICS LTD', 'UNIPOOLS LTD', 'RIVIERA HOT 

In [7]:
df["Customer name"] = df["Customer name"].str.replace(r' - 100', '')
df["Customer name"] = df["Customer name"].str.replace(r'(?!-)[^\w\s]', '', regex=True)
df["Customer name"] = df["Customer name"].str.replace('Ã‘', 'A')
df["Customer name"] = df["Customer name"].str.replace('Ã“', 'A')
df["Customer name"] = df["Customer name"].str.replace(r'[^\x00-\x7f]', '', regex=True)
df["Customer name"] = df["Customer name"].str.strip()

In [8]:
df2 = df[['Purchase Date', 'Customer name', 'Global Amount']]
df2

,Purchase Date,Customer name,Global Amount
0,03/01/23 0:00,BUCKLAND POOL BUILDING COMPANY LTD,849.73
1,03/01/23 0:00,FREEDOM LEISURE,75.02
2,03/01/23 0:00,FREEDOM LEISURE,14.6
3,03/01/23 0:00,UK POOL STORE LTD,209.89
4,03/01/23 0:00,LEISURE POOLS LTD,354.43
...,...,...,...
3060,30/06/23 0:00,RPS COMPANY SERVICES LTD,334.94
3061,30/06/23 0:00,GALA SWIMMING POOLS LTD,2148.4
3062,30/06/23 0:00,TA THE POOL CLEANERS,54.74
3063,30/06/23 0:00,HEATHCREST HEATING SERVICES LTD,701.23


In [9]:
df2["Purchase Date"] = pd.to_datetime(df2["Purchase Date"])
df2['Global Amount'] = pd.to_numeric(df2['Global Amount'])

/var/folders/yx/wg0v7ck91t14mxp5lfptvbh00000gn/T/ipykernel_5448/501058408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Purchase Date"] = pd.to_datetime(df2["Purchase Date"])
/var/folders/yx/wg0v7ck91t14mxp5lfptvbh00000gn/T/ipykernel_5448/501058408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Global Amount'] = pd.to_numeric(df2['Global Amount'])


In [10]:
df2.dtypes

Purchase Date    datetime64[ns]
Customer name            object
Global Amount           float64
dtype: object

In [11]:
snapshot_date = df2['Purchase Date'].max() + timedelta(days=1)

In [12]:
df3 = df2.groupby('Customer name').agg({'Global Amount': ['sum','count'], 'Purchase Date': lambda x: (snapshot_date - x.max()).days}).reset_index()

In [13]:
df3.head(3)

Customer name Global Amount       Purchase Date
                                     sum count      <lambda>
0           1066 POOLS LTD       1107.05     6           191
1  A  D SWIMMING POOLS LTD       7780.39    12            36
2                A A POOLS       -170.54     1           277

In [14]:
df3.columns

MultiIndex([('Customer name',         ''),
            ('Global Amount',      'sum'),
            ('Global Amount',    'count'),
            ('Purchase Date', '<lambda>')],
           )

In [15]:
df4 = df3.rename(columns={'': 'Customer Name',
                          "sum": "Monetary value",
                          'count': 'Frequency',
                          '<lambda>': 'Recency'})

In [16]:
# Dropping 1st level Index:
df4.columns = df4.columns.droplevel(0)

In [17]:
df4.head(3)

,Customer Name,Monetary value,Frequency,Recency
0,1066 POOLS LTD,1107.05,6,191
1,A D SWIMMING POOLS LTD,7780.39,12,36
2,A A POOLS,-170.54,1,277


In [18]:
df4.to_excel(f"./{month}/UK/RFM_UK_{month}.xlsx", index=False)